Pruebas en el modelo v6

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from detectors.detector_factory import detector_factory

image_ext = ['jpg', 'jpeg', 'png', 'webp']
video_ext = ['mp4', 'mov', 'avi', 'mkv']
time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']

In [2]:
sys.argv = ['-f', 'ctdetv2', '--demo', '../images/yolo/', '--arch','dlav6_34',  '--load_model' ,'../exp/ctdetv2/coco_dlav6/model_last.pth'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\default
ctdetv2


In [3]:
opt = opts().init()

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\default
heads {'hm': 80, 'wh': 2, 'reg': 2}


In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
#opt.debug = max(opt.debug, 1)
opt.debug = 0
Detector = detector_factory[opt.task]
detector = Detector(opt)

Creating model...
loaded ../exp/ctdetv2/coco_dlav6/model_last.pth, epoch 240


In [5]:
detector.model

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

In [6]:
detector.model.load_state_dict(torch.load("model_wb_10.pth"))

<All keys matched successfully>

In [7]:
class_name_caltech80 = [
      'ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'bear',
      'beer-mug', 'binoculars', 'blimp', 'bonsai', 'boom-box', 'bowling-ball',
      'bowling-pin', 'boxing-glove', 'brain', 'breadmaker', 'buddha', 'butterfly',
      'cactus', 'cake', 'calculator', 'camel', 'canoe', 'car-tire', 'cartman',
      'cereal-box', 'chess-board', 'cockroach', 'coffin', 'coin', 'computer-keyboard',
      'computer-monitor', 'computer-mouse', 'cowboy-hat', 'crab', 'desk-globe',
      'dice', 'dolphin', 'electric-guitar', 'elk', 'faces-easy', 'fire-extinguisher',
      'floppy-disk', 'frying-pan', 'gas-pump', 'golf-ball', 'grapes', 'hamburger',
      'homer-simpson', 'hot-air-ballon', 'hot-dog', 'hourglass', 'ice-cream-cone',
      'iris', 'mars', 'megaphone', 'microwave', 'motorbike', 'owl', 'photocopier',
      'picnic-table', 'playing-card', 'refrigerator', 'revolver', 'saturn', 'snail',
      'soccer-ball', 'soda-can', 'spaghetti', 'spider', 'starfish', 'steering-wheel',
      'swan', 'swiss-army-knife', 'tennis-ball', 'tennis-racket', 'toaster', 'top-hat',
      'vcr','washing-machine']

In [8]:
opt.demo = '../images/256_ObjectCategories/Caltech80/train/'

images_path = {}
ls = os.listdir(opt.demo)
image_names = []
ls = os.listdir(opt.demo)
for folder_name in sorted(ls):
    image_names = []
    ls2 = os.listdir(os.path.join(opt.demo, folder_name))
    for file_name in sorted(ls2):
      ext = file_name[file_name.rfind('.') + 1:].lower()
      if ext in image_ext:
          image_names.append(os.path.join(opt.demo, folder_name, file_name).replace("\\","/"))
    images_path[folder_name] = image_names


In [9]:
count = 0
total = 0
for index_w, folder in enumerate(images_path.keys()): 
    for (image_name) in images_path[folder]:
        ret = detector.run(image_name)
        dets = ret['dets']
        topC = int(dets[0,0,5])
        if(folder == class_name_caltech80[topC] ):
            count+=1
        total += 1
acc = count / total
print(acc)

0.36
